In [1]:
import pandas as pd
import numpy as np
from functools import reduce
import time
pd.set_option("display.max_columns", None)

In [11]:
# function which will take two words as paramaters, and return the sorted words in a list

def sortWords(s1, s2):
    words = [s1, s2]
    words.sort()
    
    return (words[0], words[1])

In [17]:
# function will take the df of a single team and compute the average over the past 10 games for every datapoint
# first game of each season will be replaced with 0, and until 10 games have been played we will record the average of the
# games played up to that point in the season


def get10GameAveragesPerSeasonNo0s(df):
    df = df.sort_values(by='GameID')
    
    cols = list(df.columns)[4:]
    
    for index,col in enumerate(cols):
        previousTen = []
        gameCount = 11
        currentSeason = ''
        for idx, item in enumerate(df[col]):
            if gameCount < 10:
                df.iloc[idx,index+4] = np.mean(previousTen)
                previousTen.append(item)
                gameCount +=1
            elif currentSeason == df.iloc[idx,2]:
                df.iloc[idx,index+4] = np.mean(previousTen)
                del previousTen[0]
                previousTen.append(item)
            else:
                currentSeason == df.iloc[idx,2]
                gameCount = 1
                df.iloc[idx,index+4] = 0
                previousTen = [item]
    return df

In [12]:
# assigns 10 game averages and carries over the averages from the end of the previous season

def get10GameAverages(df):
    df = df.sort_values(by='GameID')
    
    cols = list(df.columns)[4:]
    
    for index,col in enumerate(cols):
        previousTen = [0]
        for idx, item in enumerate(df[col]):
            if idx<10:
                df.iloc[idx,index+4] = np.mean(previousTen)
                previousTen.append(item)
            else:
                df.iloc[idx,index+4] = np.mean(previousTen)
                del previousTen[0]
                previousTen.append(item)
    return df


In [13]:
# this function will take the homeAway dataframe and clean it, more specifically it will add the gameID column,
# fix a couple columns, and sort the dataframe

def sortHomeAway(df):
    newDates = []
    gameIdVals = []
    dates = list(df.loc[:,'Date'])
    home = list(df.loc[:,'Home Team '])
    away = list(df.loc[:,'Away Team'])
    
    for i, date in enumerate(dates):
        newDates.append(date.replace('/','-'))
        if home[i] == 'CHH':
            home[i] = 'CHA'
        elif home[i] == 'NJN':
            home[i] = 'BKN'
        elif home[i] == 'NOK' or home[i] == 'NOH':
            home[i] = 'NOP'
        elif home[i] == 'SEA':
            home[i] = 'OKC'
        elif home[i] == 'VAN':
            home[i] = 'MEM'
            
        if away[i] == 'CHH':
            away[i] = 'CHA'
        elif away[i] == 'NJN':
            away[i] = 'BKN'
        elif away[i] == 'NOK' or away[i] == 'NOH':
            away[i] = 'NOP'
        elif away[i] == 'SEA':
            away[i] = 'OKC'
        elif away[i] == 'VAN':
            away[i] = 'MEM'
            
        words = sortWords(home[i],away[i])
        gameIdVals.append(newDates[i]+words[0]+'v'+words[1])
        
    df.insert(loc=1,column='GameID',value=gameIdVals)
    df.loc[:,'Date'] = pd.Series(newDates)
    df.loc[:,'Home Team '] = pd.Series(home)
    df.loc[:,'Away Team'] = pd.Series(away)
    df.rename(columns={'Home Team ':'Home Team'},inplace=True)
    sortedFrame = df.sort_values(by='GameID').reset_index(drop=True)
    
    
    return sortedFrame

In [14]:
# This box should create a combineTeamData function
# This function should take in a list of dataframes for a team, and combine them into 1 team
# Additionally, this function should create a new column 'GameID' of the form: 'dateTeamvTeam'
# The function should also create a season column, which specifies the season that the statistics come from
# Finally, this function should make sure that the opponent column is cleaned up and in the right posistion

def combineTeamData(theList, team):
    
    # this line combines the five datsets into one
    df = reduce(lambda left, right: pd.merge(left,right,on=['Date'],how='outer'),theList)
    
    # this line moves the opponent column to the correct spot
    df.insert(1, 'Opponent', df.pop('Opponent'))
    
    # this block deletes all the unnecessary opponent columns
    daList = []
    for i in list(df.columns):
        if 'Opponent_' in i:
            daList.append(i)
    df.drop(daList, axis=1, inplace=True)  
    
    # this block will create the gameID and season columns
    gameIDVals = []
    seasonVals = []
    wrongDates = []
    dates = list(df.loc[:,'Date'])
    opponent = list(df.loc[:,'Opponent'])
    date1 = pd.Timestamp('2021-10-16')
    date2 = pd.Timestamp('2022-09-01')

    for i, date in enumerate(dates):
        if opponent[i] == 'CHH':
            opponent[i] = 'CHA'
        elif opponent[i] == 'NJN':
            opponent[i] = 'BKN'
        elif opponent[i] == 'NOK':
            opponent[i] = 'NOP'
        elif opponent[i] == 'SEA':
            opponent[i] = 'OKC'
        dates[i] = str(dates[i])[:-9]
        words = sortWords(team,opponent[i])
        gameIDVals.append(dates[i]+words[0]+'v'+words[1])
        string = ''
        year = str(date.year)[2:]
        year1 = str(int(date.year)+1)[2:]
        year2 = str(int(date.year)-1)[2:]
        if date.month >= date1.month:
            string += year+'-'+year1
        else:
            string += year2+'-'+year
        seasonVals.append(string)
    
    df.loc[:, 'Opponent'] = pd.Series(opponent)    
    df.insert(loc=1,column='GameID',value=gameIDVals)
    df.insert(loc=2,column='Season',value=seasonVals)
    df.drop('ShotQualityAvg_y', axis=1,inplace=True)
    df.rename(columns={'ShotQualityAvg_x':'ShotQualityAvg'},inplace=True)
    
    
    return df

In [26]:
# this function takes a list of teams, download their data, call combineTeamData(), and add it to a list 
# bottom block is print statements to display download time, will return list of team dataframes
# can call an averages function to compute the averages if necessary

def combineAllTeams(listOfTeams):
    teamList = []
    cumulative = time.time()
    
    # this block will loop through the teams, and call combineTeamData()
    for i, j in enumerate(listOfTeams):
        begin = time.time()
        string = "C:\\Users\\marte\\OneDrive\\Documents\\Projects!\\NBA Model\\Data\\"
        assists = pd.read_excel(string + j + 'SCORING.xlsx')
        rebound = pd.read_excel(string + j + 'SHOOTING.xlsx')
        scoring = pd.read_excel(string + j + 'ASSISTS.xlsx')
        shooting = pd.read_excel(string + j + 'REBOUND.xlsx')
        turnover = pd.read_excel(string + j + 'TURNOVER.xlsx')
        
        listPerTeam = [assists,rebound,scoring,shooting,turnover]
        teamDf = combineTeamData(listPerTeam,j)
        
        teamDf = get10GameAveragesPerSeason(teamDf)
        
        teamList.append(teamDf)
        
        end = time.time()
        passed = str(round(end-begin,2))
        
        print(' '+j+' TEAM DATA COMBINED')
        print(' '+str(i+1)+' / 30 TOTAL TEAMS')
        if int(round(end-begin,2))>60:
            newPassed = str(round((end-begin)/60,2))
            print(' '+j+' DOWNLOAD TIME: '+newPassed+' MINUTES')
        else:
            print(' '+j+' DOWNLOAD TIME: '+passed+' SECONDS')
        if int(end-cumulative)>60:
            totalPassed = str(round((end-cumulative)/60,2))
            print(' TOTAL TIME PASSED: '+totalPassed+' MINUTES')
        else:
            totalPassed = str(round(end-cumulative,2))
            print(' TOTAL TIME PASSED: '+totalPassed+' SECONDS')
        print('|---------------------------------|')
        
    print('COMBINING ALL DATAFRAMES.....')
    df = pd.concat(teamList)
    df = df.sort_values(by='GameID')
    print('DONE')
    
    
    return df


In [16]:
# this function will take the dataframes of all teams and combine them into 1
# each game will be in there twice, so we will need combine those games into a single row
# f will have to change the name of the opponents columns to 'XXstat'
# f will also add Home/Away columns for clarity/the model, and ensure every home team is on the left


def combineEachGame(sortedFrame):
    
    # this block combines the data, sorts it by gameID, and combines the two rows of each game into one
    # 'XX' before a stat will indicate the away team
    df1 = sortedFrame.iloc[::2, :]
    df2 = sortedFrame.iloc[1::2, :]
    df1.reset_index(inplace=True, drop=True)
    df2.reset_index(inplace=True, drop=True)
    
    for i in df2.columns:
        df2.rename(columns={i:'XX'+i},inplace=True)
    df = pd.concat([df1, df2], join='outer', axis=1)
    df.drop(['XXDate','XXGameID','XXSeason'],axis=1,inplace=True)
    
    # this block adds a Home/Away column with 1 = home and 0 = away
    haList = []
    XXhaList = []
    
    for i,v in enumerate(sortedHomeAway['Away Team']):
        if v != df['Opponent'][i]:
            haList.append(0)
            XXhaList.append(1)
        else:
            haList.append(1)
            XXhaList.append(0)
    df.insert(loc=3,column='HomeAway',value=haList)
    df.insert(loc=124,column='XXHomeAway',value=XXhaList)            
    
    # this portion cleans the df and ensures every home team is on the left
    dfColumns = list(df.columns)
    df3 = dfColumns[3:124]
    df4 = dfColumns[124:]
    left = np.asanyarray(df[df['HomeAway']==0].loc[:,df3])
    right = np.asanyarray(df[df['HomeAway']==0].loc[:,df4])
    df.loc[df['HomeAway'] == 0,df4] = left    
    df.loc[df['HomeAway'] == 0,df3] = right

    part1 = list(df['XXOpponent'])
    part2 = list(df['Opponent'])
    df.loc[:,'Opponent'] = part1
    df.loc[:,'XXOpponent'] = part2
    df.rename(columns={'Opponent':'HomeTeam','XXOpponent':'XXAwayTeam'},inplace=True)

    
    return df

In [17]:
homeAway = pd.read_excel(r'C:\Users\marte\OneDrive\Documents\Projects!\NBA Model\Data\1HOME-AWAY.xlsx')
sortedHomeAway = sortHomeAway(homeAway)
sortedHomeAway

,Date,GameID,Home Team,Away Team,Home Points,Away Points,Home Possessions,Away Possessions
0,2000-10-31,2000-10-31ATLvCHA,ATL,CHA,82,106,91,92
1,2000-10-31,2000-10-31BKNvCLE,BKN,CLE,82,86,96,96
2,2000-10-31,2000-10-31CHIvSAC,CHI,SAC,81,100,91,90
3,2000-10-31,2000-10-31DALvMIL,DAL,MIL,97,93,97,97
4,2000-10-31,2000-10-31DETvTOR,TOR,DET,95,104,99,100
...,...,...,...,...,...,...,...,...
26329,2022-04-10,2022-04-10LACvOKC,LAC,OKC,138,88,98,98
26330,2022-04-10,2022-04-10MIAvORL,ORL,MIA,125,111,98,98
26331,2022-04-10,2022-04-10NYKvTOR,NYK,TOR,105,94,94,95
26332,2022-04-10,2022-04-10PHXvSAC,PHX,SAC,109,116,98,98


In [27]:
listOfTeams = ['ATL','BKN','BOS','CHA','CHI','CLE','DAL','DEN','DET','GSW','HOU','IND',
               'LAC','LAL','MEM','MIA','MIL','MIN','NOP','NYK','OKC','ORL','PHI','PHX',
               'POR','SAC','SAS','TOR','UTA','WAS']

allTeams = combineAllTeams(listOfTeams)

 ATL TEAM DATA COMBINED
 1 / 30 TOTAL TEAMS
 ATL DOWNLOAD TIME: 56.92 SECONDS
 TOTAL TIME PASSED: 56.92 SECONDS
|---------------------------------|
 BKN TEAM DATA COMBINED
 2 / 30 TOTAL TEAMS
 BKN DOWNLOAD TIME: 56.88 SECONDS
 TOTAL TIME PASSED: 1.9 MINUTES
|---------------------------------|
 BOS TEAM DATA COMBINED
 3 / 30 TOTAL TEAMS
 BOS DOWNLOAD TIME: 1.13 MINUTES
 TOTAL TIME PASSED: 3.03 MINUTES
|---------------------------------|
 CHA TEAM DATA COMBINED
 4 / 30 TOTAL TEAMS
 CHA DOWNLOAD TIME: 1.25 MINUTES
 TOTAL TIME PASSED: 4.28 MINUTES
|---------------------------------|
 CHI TEAM DATA COMBINED
 5 / 30 TOTAL TEAMS
 CHI DOWNLOAD TIME: 1.11 MINUTES
 TOTAL TIME PASSED: 5.39 MINUTES
|---------------------------------|
 CLE TEAM DATA COMBINED
 6 / 30 TOTAL TEAMS
 CLE DOWNLOAD TIME: 58.05 SECONDS
 TOTAL TIME PASSED: 6.36 MINUTES
|---------------------------------|
 DAL TEAM DATA COMBINED
 7 / 30 TOTAL TEAMS
 DAL DOWNLOAD TIME: 57.93 SECONDS
 TOTAL TIME PASSED: 7.32 MINUTES
|---------

In [28]:
final = combineEachGame(allTeams)
final

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,Date,GameID,Season,HomeAway,HomeTeam,OffPoss,Points,FG2M,FG2A,Fg2Pct,FG3M,FG3A,Fg3Pct,NonHeaveFg3Pct,FtPoints,PtsAssisted2s,PtsUnassisted2s,PtsAssisted3s,PtsUnassisted3s,Assisted2sPct,NonPutbacksAssisted2sPct,Assisted3sPct,FG3APct,ShotQualityAvg,EfgPct,TsPct,PtsPutbacks,Fg2aBlocked,FG2APctBlocked,Fg3aBlocked,FG3APctBlocked,AtRimFG3AFrequency,Avg2ptShotDistance,Avg3ptShotDistance,AtRimFGM,AtRimFGA,AtRimFrequency,AtRimAccuracy,UnblockedAtRimAccuracy,AtRimPctAssisted,AtRimPctBlocked,ShortMidRangeFGM,ShortMidRangeFGA,ShortMidRangeFrequency,ShortMidRangeAccuracy,UnblockedShortMidRangeAccuracy,ShortMidRangePctAssisted,ShortMidRangePctBlocked,LongMidRangeFGM,LongMidRangeFGA,LongMidRangeFrequency,LongMidRangeAccuracy,UnblockedLongMidRangeAccuracy,LongMidRangePctAssisted,LongMidRangePctBlocked,Corner3FGM,Corner3FGA,Corner3Frequency,Corner3Accuracy,UnblockedCorner3Accuracy,Corner3PctAssisted,Corner3PctBlocked,Arc3FGM,Arc3FGA,Arc3Frequency,Arc3Accuracy,UnblockedArc3Accuracy,Arc3PctAssisted,Arc3PctBlocked,NonHeaveArc3FGM,NonHeaveArc3FGA,NonHeaveArc3Accuracy,HeaveAttempts,HeaveMakes,Assists,AssistPoints,TwoPtAssists,ThreePtAssists,AtRimAssists,ShortMidRangeAssists,LongMidRangeAssists,Corner3Assists,Arc3Assists,Rebounds,DefRebounds,FTDefRebounds,DefFTReboundPct,DefTwoPtRebounds,DefTwoPtReboundPct,DefThreePtRebounds,DefThreePtReboundPct,DefFGReboundPct,OffRebounds,FTOffRebounds,OffFTReboundPct,OffTwoPtRebounds,OffTwoPtReboundPct,OffThreePtRebounds,OffThreePtReboundPct,OffFGReboundPct,DefAtRimReboundPct,DefShortMidRangeReboundPct,DefLongMidRangeReboundPct,DefArc3ReboundPct,DefCorner3ReboundPct,OffAtRimReboundPct,OffShortMidRangeReboundPct,OffLongMidRangeReboundPct,OffArc3ReboundPct,OffCorner3ReboundPct,SelfOReb,SelfORebPct,Turnovers,LiveBallTurnovers,DeadBallTurnovers,LiveBallTurnoverPct,LostBallTurnovers,LostBallOutOfBoundsTurnovers,BadPassTurnovers,BadPassOutOfBoundsTurnovers,Travels,3SecondViolations,StepOutOfBoundsTurnovers,OffensiveGoaltends,XXHomeAway,XXAwayTeam,XXOffPoss,XXPoints,XXFG2M,XXFG2A,XXFg2Pct,XXFG3M,XXFG3A,XXFg3Pct,XXNonHeaveFg3Pct,XXFtPoints,XXPtsAssisted2s,XXPtsUnassisted2s,XXPtsAssisted3s,XXPtsUnassisted3s,XXAssisted2sPct,XXNonPutbacksAssisted2sPct,XXAssisted3sPct,XXFG3APct,XXShotQualityAvg,XXEfgPct,XXTsPct,XXPtsPutbacks,XXFg2aBlocked,XXFG2APctBlocked,XXFg3aBlocked,XXFG3APctBlocked,XXAtRimFG3AFrequency,XXAvg2ptShotDistance,XXAvg3ptShotDistance,XXAtRimFGM,XXAtRimFGA,XXAtRimFrequency,XXAtRimAccuracy,XXUnblockedAtRimAccuracy,XXAtRimPctAssisted,XXAtRimPctBlocked,XXShortMidRangeFGM,XXShortMidRangeFGA,XXShortMidRangeFrequency,XXShortMidRangeAccuracy,XXUnblockedShortMidRangeAccuracy,XXShortMidRangePctAssisted,XXShortMidRangePctBlocked,XXLongMidRangeFGM,XXLongMidRangeFGA,XXLongMidRangeFrequency,XXLongMidRangeAccuracy,XXUnblockedLongMidRangeAccuracy,XXLongMidRangePctAssisted,XXLongMidRangePctBlocked,XXCorner3FGM,XXCorner3FGA,XXCorner3Frequency,XXCorner3Accuracy,XXUnblockedCorner3Accuracy,XXCorner3PctAssisted,XXCorner3PctBlocked,XXArc3FGM,XXArc3FGA,XXArc3Frequency,XXArc3Accuracy,XXUnblockedArc3Accuracy,XXArc3PctAssisted,XXArc3PctBlocked,XXNonHeaveArc3FGM,XXNonHeaveArc3FGA,XXNonHeaveArc3Accuracy,XXHeaveAttempts,XXHeaveMakes,XXAssists,XXAssistPoints,XXTwoPtAssists,XXThreePtAssists,XXAtRimAssists,XXShortMidRangeAssists,XXLongMidRangeAssists,XXCorner3Assists,XXArc3Assists,XXRebounds,XXDefRebounds,XXFTDefRebounds,XXDefFTReboundPct,XXDefTwoPtRebounds,XXDefTwoPtReboundPct,XXDefThreePtRebounds,XXDefThreePtReboundPct,XXDefFGReboundPct,XXOffRebounds,XXFTOffRebounds,XXOffFTReboundPct,XXOffTwoPtRebounds,XXOffTwoPtReboundPct,XXOffThreePtRebounds,XXOffThreePtReboundPct,XXOffFGReboundPct,XXDefAtRimReboundPct,XXDefShortMidRangeReboundPct,XXDefLongMidRangeReboundPct,XXDefArc3ReboundPct,XXDefCorner3ReboundPct,XXOffAtRimReboundPct,XXOffShortMidRangeReboundPct,XXOffLongMidRangeReboundPct,XXOffArc3ReboundPct,XXOffCorner3ReboundPct,XXSelfOReb,XXSelfORebPct,XXTurnovers,XXLiveBallTurnovers,XXDeadBallTurnovers,XXLiveBallTurnoverPct,XXLostBallTurnovers,XXLo

In [29]:
final.to_csv(r'C:\Users\marte\OneDrive\Documents\Projects!\NBA Model\cleanDataAvgs.csv', index=False)

In [ ]:
# this function will check that the current dataframe is correct, and there were no mistakes in cleaning
# we will check that all the teams are in the right spots, and the associated stats are all correct, we will check
# the gameID and season columns, and we will select random rows / columns / teams to ensure accuracy


# functiion which counts the number of games in each season for a specific team's season column

def numOfEach(theList):
    numOf = []
    uhOh = []
    current = '00-01'
    counter = 1
    for i in theList:
        if i == current:
            counter+=1
        elif i == '|--------|':
            uhOh.append(i)
        else:
            numOf.append(current+': '+str(counter))
            current = i
            counter = 1
    numOf.append(current+': '+str(counter))
    print(numOf)
